In [2]:
from dotenv import dotenv_values
env_var = dotenv_values(".env")
env_var["MINIO_ROOT_USER"]

'minio'

In [3]:
from dags.module.object_client import MinioClient
minio_client = MinioClient("localhost", "9000", access_key=env_var["MINIO_ROOT_USER"], secret_key=env_var["MINIO_ROOT_PASSWORD"])
minio_client.load_object("bronze", "data_TSCO.LON_bronze.json", "data.json")
response = minio_client.get_object("silver", "data_TSCO.LON_silver.csv")

import pandas as pd
df = pd.read_csv(response)
df

KeyboardInterrupt: 

In [18]:
data = minio_client.get_object("bronze", "data_TSCO.LON_bronze.json")
data

{'2024-07-25': {'1. open': '325.6000',
  '2. high': '327.0980',
  '3. low': '323.4000',
  '4. close': '325.6000',
  '5. volume': '7837640'},
 '2024-07-24': {'1. open': '322.9000',
  '2. high': '327.3000',
  '3. low': '321.8000',
  '4. close': '327.2000',
  '5. volume': '12812040'},
 '2024-07-23': {'1. open': '326.5000',
  '2. high': '327.0000',
  '3. low': '324.6822',
  '4. close': '325.9000',
  '5. volume': '24617221'},
 '2024-07-22': {'1. open': '326.8000',
  '2. high': '327.8000',
  '3. low': '325.0000',
  '4. close': '326.2000',
  '5. volume': '7915016'},
 '2024-07-19': {'1. open': '324.1000',
  '2. high': '326.0000',
  '3. low': '319.6000',
  '4. close': '326.0000',
  '5. volume': '10216170'},
 '2024-07-18': {'1. open': '323.5000',
  '2. high': '324.9000',
  '3. low': '319.6000',
  '4. close': '324.9000',
  '5. volume': '19339840'},
 '2024-07-17': {'1. open': '318.4000',
  '2. high': '321.6000',
  '3. low': '316.3000',
  '4. close': '321.6000',
  '5. volume': '31080090'},
 '2024-0

In [19]:
import pandas as pd
df = pd.DataFrame.from_dict(data, orient='index').reset_index()
df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
df['Date'] = pd.to_datetime(df['Date'])
df

,Date,Open,High,Low,Close,Volume
0,2024-07-25,325.6000,327.0980,323.4000,325.6000,7837640
1,2024-07-24,322.9000,327.3000,321.8000,327.2000,12812040
2,2024-07-23,326.5000,327.0000,324.6822,325.9000,24617221
3,2024-07-22,326.8000,327.8000,325.0000,326.2000,7915016
4,2024-07-19,324.1000,326.0000,319.6000,326.0000,10216170
...,...,...,...,...,...,...
4937,2005-01-10,319.7501,320.5001,319.0001,320.2501,11117302
4938,2005-01-07,317.7501,321.2501,317.7501,319.7501,37111214
4939,2005-01-06,315.0001,320.0001,314.7501,318.7501,17598422
4940,2005-01-05,315.5001,318.0001,315.0001,315.0001,28205028


In [20]:
object_data = minio_client.get_object("bronze", 'data_TSCO.LON_bronze.json')
df = pd.DataFrame.from_dict(object_data, orient='index').reset_index()
df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
df['Date'] = pd.to_datetime(df['Date'])

In [21]:
import boto3
from io import StringIO
access_key = "heGJDX8PMgKFKOwXIKdN"
secret_key = "8lgtuhi2Zkn51mWkHEILIVwJMkcb049QLs3534wY"
csv_buffer = StringIO()
df.to_csv(csv_buffer, sep=",")
s3_resource = boto3.resource(
        's3',
        endpoint_url="http://localhost:9000",
        aws_access_key_id=access_key,
        aws_secret_access_key=secret_key,
        config=boto3.session.Config(signature_version='s3v4')
        )
s3_resource.Object("silver", 'data_TSCO.LON_silver.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '17E5B96274DA7D75',
  'HostId': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '0',
   'etag': '"3730795c08dd45b4ff7e5df6f5c45aff"',
   'server': 'MinIO',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
   'x-amz-request-id': '17E5B96274DA7D75',
   'x-content-type-options': 'nosniff',
   'x-xss-protection': '1; mode=block',
   'date': 'Fri, 26 Jul 2024 09:31:05 GMT'},
  'RetryAttempts': 0},
 'ETag': '"3730795c08dd45b4ff7e5df6f5c45aff"'}

In [1]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    database="userdb",
    user="username",
    password="password",
    port=5439,
)

# Query to get the data
sql = '''
    SELECT * FROM public."IBM";
'''
# Cursor
cursor = conn.cursor()

# Execute the query
cursor.execute(sql)

# Query result
data = cursor.fetchall()

# Convert data to pandas dataframe
df = pd.DataFrame(data, columns=[desc[0] for desc in cursor.description])

df

OperationalError: connection to server at "localhost" (::1), port 5439 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5439 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
